# [Writing Workflows](https://www.commonwl.org/user_guide/21-1st-workflow/index.html)

<div class="container" style="background-color: rgba(128, 222, 234, 0.265); border-radius: 12px; padding: 8px; margin-bottom: 8px;">
    <h2 style="padding: 0px; margin-top: 0px">Overview</h2>
    <ul>
        <li><b>Teaching:</b> 10 min</li>
        <li><b>Exercises:</b> 0 min</li>
    </ul>
    <h3 style="padding: 0px; margin-top: 0px">Questions</h3>
    <ul>
        <li>"How do I connect tools together into a workflow?"</li>
    </ul>
    <h3 style="padding: 0px; margin-top: 0px">Objectives</h3>
    <ul>
        <li>"Learn how to construct workflows from multiple CWL tool descriptions."</li>
    </ul>
</div>

This workflow extracts a java source file from a tar file and then compiles it.

In [ ]:
cwlVersion: v1.0
class: CommandLineTool
id: tar-param
baseCommand: [tar, --extract]
inputs:
  tarfile:
    type: File
    inputBinding:
      prefix: --file
  extractfile:
    type: string
    inputBinding:
      position: 1
outputs:
  extracted_file:
    type: File
    outputBinding:
      glob: $(inputs.extractfile)

In [ ]:
cwlVersion: v1.0
class: CommandLineTool
id: arguments
label: Example trivial wrapper for Java 9 compiler
hints:
  DockerRequirement:
    dockerPull: openjdk:9.0.1-11-slim
baseCommand: javac
arguments: ["-d", $(runtime.outdir)]
inputs:
  src:
    type: File
    inputBinding:
      position: 1
outputs:
  classfile:
    type: File
    outputBinding:
      glob: "*.class"

In [ ]:
% snippet add
cwlVersion: v1.0
class: Workflow
id: 1st-workflow

The `cwlVersion` field indicates the version of the CWL spec used by the document. The `class` field indicates this document describes a workflow.

In [4]:
% snippet add
inputs:
  tarball: 
    type: File
  name_of_file_to_extract: 
    type: string

{"cwlVersion": "v1.0", "class": "Workflow", "id": "1st-workflow", "inputs": {"tarball": {"type": "File"}, "name_of_file_to_extract": {"type": "string"}}}

The `inputs` section describes the inputs of the workflow. This is a list of input parameters where each parameter consists of an identifier and a data type. These parameters can be used as sources for input to specific workflows steps.

In [5]:
% snippet add
outputs:
  compiled_class:
    type: File
    outputSource: compile/classfile

{"cwlVersion": "v1.0", "class": "Workflow", "id": "1st-workflow", "inputs": {"tarball": {"type": "File"}, "name_of_file_to_extract": {"type": "string"}}, "outputs": {"compiled_class": {"type": "File", "outputSource": "compile/classfile"}}}

The `outputs` section describes the outputs of the workflow. This is a list of output parameters where each parameter consists of an identifier and a data type. The `outputSource` connects the output parameter `classfile` of the `compile` step to the workflow output parameter `compiled_class`.

In [6]:
% snippet add
steps:
  untar:
    run: tar-param.cwl
    in:
      tarfile: tarball
      extractfile: name_of_file_to_extract
    out: [extracted_file]

{"cwlVersion": "v1.0", "class": "Workflow", "id": "1st-workflow", "inputs": {"tarball": {"type": "File"}, "name_of_file_to_extract": {"type": "string"}}, "outputs": {"compiled_class": {"type": "File", "outputSource": "compile/classfile"}}, "steps": {"untar": {"run": "tar-param.cwl", "in": {"tarfile": "tarball", "extractfile": "name_of_file_to_extract"}, "out": ["extracted_file"]}}}

The `steps` section describes the actual steps of the workflow. In this example, the first step extracts a file from a tar file, and the second step compiles the file from the first step using the java compiler. Workflow steps are not necessarily run in the order they are listed, instead the order is determined by the dependencies between steps (using `source`). In addition, workflow steps which do not depend on one another may run in parallel.

The first step, `untar` runs `tar-param.cwl` (described previously in [Parameter references](Parameter%20References.ipynb). This tool has two input parameters, `tarfile` and `extractfile` and one output parameter `extracted_file`.

The in section of the workflow step connects these two input parameters to the inputs of the workflow, `tarball` and `name_of_file_to_extract` using `source`. This means that when the workflow step is executed, the values assigned to `tarball` and `name_of_file_to_extract` will be used for the parameters `tarfile` and `extractfile` in order to run the tool.

The `out` section of the workflow step lists the output parameters that are expected from the tool.

In [7]:
% snippet build
  compile:
    run: arguments.cwl
    in:
      src: untar/extracted_file
    out: [classfile]

{"cwlVersion": "v1.0", "class": "Workflow", "id": "1st-workflow", "inputs": {"tarball": {"type": "File"}, "name_of_file_to_extract": {"type": "string"}}, "outputs": {"compiled_class": {"type": "File", "outputSource": "compile/classfile"}}, "steps": {"untar": {"run": "tar-param.cwl", "in": {"tarfile": "tarball", "extractfile": "name_of_file_to_extract"}, "out": ["extracted_file"]}, "compile": {"run": "arguments.cwl", "in": {"src": "untar/extracted_file"}, "out": ["classfile"]}}}

The second step `compile` depends on the results from the first step by connecting the input parameter `src` to the output parameter of `untar` using `untar/extracted_file`. The output of this step `classfile` is connected to the `outputs` section for the Workflow, described above.

In [8]:
% view 1st-workflow

<div style="max-width: 100%;"><svg xmlns="http://www.w3.org/2000/svg" width="420pt" height="320pt" viewBox="0.00 0.00 420.00 320.00">
<g id="graph0" class="graph" transform="scale(1 1) rotate(0) translate(4 316)">
<title>G</title>
<polygon fill="#eeeeee" stroke="transparent" points="-4,4 -4,-316 416,-316 416,4 -4,4"></polygon>
<g id="clust1" class="cluster">
<title>cluster_inputs</title>
<polygon fill="#eeeeee" stroke="black" stroke-dasharray="5,2" points="8,-229 8,-304 404,-304 404,-229 8,-229"></polygon>
<text text-anchor="middle" x="348.92" y="-235.8" font-family="Times,serif" font-size="14.00">Workflow Inputs</text>
</g>
<g id="clust2" class="cluster">
<title>cluster_outputs</title>
<polygon fill="#eeeeee" stroke="black" stroke-dasharray="5,2" points="121,-8 121,-83 317,-83 317,-8 121,-8"></polygon>
<text text-anchor="middle" x="257.25" y="-14.8" font-family="Times,serif" font-size="14.00">Workflow Outputs</text>
</g>

<g id="node1" class="node">
<title>file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/repo/1st-workflow.cwl#1st-workflow/untar</title>
<polygon fill="lightgoldenrodyellow" stroke="black" points="157.34,-184.5 157.34,-220.5 280.66,-220.5 280.66,-184.5 157.34,-184.5"></polygon>
<text text-anchor="middle" x="219" y="-198.3" font-family="Times,serif" font-size="14.00">1st-workflow/untar</text>
</g>

<g id="node2" class="node">
<title>file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/repo/1st-workflow.cwl#1st-workflow/compile</title>
<polygon fill="lightgoldenrodyellow" stroke="black" points="86.99,-111.5 86.99,-147.5 351.01,-147.5 351.01,-111.5 86.99,-111.5"></polygon>
<text text-anchor="middle" x="219" y="-125.3" font-family="Times,serif" font-size="14.00">Example trivial wrapper for Java 9 compiler</text>
</g>

<g id="edge1" class="edge">
<title>file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/repo/1st-workflow.cwl#1st-workflow/untar-&gt;file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/repo/1st-workflow.cwl#1st-workflow/compile</title>
<path fill="none" stroke="black" d="M219,-184.31C219,-176.29 219,-166.55 219,-157.57"></path>
<polygon fill="black" stroke="black" points="222.5,-157.53 219,-147.53 215.5,-157.53 222.5,-157.53"></polygon>
</g>

<g id="node5" class="node">
<title>file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/repo/1st-workflow.cwl#1st-workflow/compiled_class</title>
<polygon fill="#94ddf4" stroke="black" points="128.57,-38.5 128.57,-74.5 309.43,-74.5 309.43,-38.5 128.57,-38.5"></polygon>
<text text-anchor="middle" x="219" y="-52.3" font-family="Times,serif" font-size="14.00">1st-workflow/compiled_class</text>
</g>

<g id="edge4" class="edge">
<title>file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/repo/1st-workflow.cwl#1st-workflow/compile-&gt;file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/repo/1st-workflow.cwl#1st-workflow/compiled_class</title>
<path fill="none" stroke="black" d="M219,-111.31C219,-103.29 219,-93.55 219,-84.57"></path>
<polygon fill="black" stroke="black" points="222.5,-84.53 219,-74.53 215.5,-84.53 222.5,-84.53"></polygon>
</g>

<g id="node3" class="node">
<title>file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/repo/1st-workflow.cwl#1st-workflow/tarball</title>
<polygon fill="#94ddf4" stroke="black" points="265.84,-259.5 265.84,-295.5 396.16,-295.5 396.16,-259.5 265.84,-259.5"></polygon>
<text text-anchor="middle" x="331" y="-273.3" font-family="Times,serif" font-size="14.00">1st-workflow/tarball</text>
</g>

<g id="edge2" class="edge">
<title>file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/repo/1st-workflow.cwl#1st-workflow/tarball-&gt;file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/repo/1st-workflow.cwl#1st-workflow/untar</title>
<path fill="none" stroke="black" d="M304.74,-259.38C289.58,-249.5 270.31,-236.94 253.88,-226.24"></path>
<polygon fill=

In [9]:
% system echo "public class Hello {}" > Hello.java && tar -cvf hello.tar Hello.java
% execute 1st-workflow
tarball:
  class: File
  location: hello.tar
name_of_file_to_extract: Hello.java

a Hello.java


Add data in memory

{"compiled_class": {"location": "file:///private/tmp/CWLKERNEL_DATA/b8478e7e-bd68-48a4-a304-97d9b3597366/runtime_data/Hello.class", "basename": "Hello.class", "nameroot": "Hello", "nameext": ".class", "class": "File", "checksum": "sha1$fdb876b40ad9ebc7fee873212e02d5940588642e", "size": 184, "http://commonwl.org/cwltool#generation": 0, "_produced_by": "1st-workflow", "id": "compiled_class", "result_counter": 0}}

<div class="container" style="background-color: rgba(129, 199, 132, 0.265); border-radius: 12px; padding: 8px">
    <h2 style="padding: 0px; margin-top: 0px">Key Points</h2>
    <ul>
        <li>"Each step in a workflow must have its own CWL description."</li>
        <li>"Top level <code>inputs</code> and <code>outputs</code> of the workflow are described in the inputs and outputs fields respectively."</li>
        <li>"The steps are specified under <code>steps</code>."</li>
        <li>"Execution order is determined by the connections between steps."</li>
    </ul>
</div>